In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from pykrx import stock
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def init_driver(headless=False, download_dir=os.getcwd()):
    service = Service(ChromeDriverManager().install())
    chrome_options = Options()

    if headless:
        chrome_options.add_argument("--headless")

    # 기본 다운로드 경로 설정
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,  # 다운로드 시 자동으로 처리
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True  # PDF 파일을 자동으로 다운로드하도록 설정
    }
    chrome_options.add_experimental_option("prefs", prefs)
    
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=service, options=chrome_options)

    return driver

def get_latest_file(download_path):
    """Download path에서 가장 최근에 생성된 파일을 반환"""
    files = [f for f in os.listdir(download_path) if f.endswith(".pdf")]
    if not files:
        return None
    latest_file = max([os.path.join(download_path, f) for f in files], key=os.path.getctime)
    return latest_file


def get_ticker_by_name(stock_name: str) -> str:
    """
    입력된 종목 이름에 해당하는 티커를 반환합니다.
    
    :param stock_name: 조회할 종목 이름
    :return: 종목에 해당하는 티커 또는 None (종목이 존재하지 않으면)
    """
    print('종목명: ',stock_name)
    try:
        # 모든 티커 리스트를 가져오기
        tickers = stock.get_market_ticker_list()

        # 티커 리스트를 순회하면서 종목 이름과 매칭되는 티커 찾기
        for ticker in tickers:
            ticker_name = stock.get_market_ticker_name(ticker)
            if ticker_name and ticker_name.strip() == stock_name.strip():
                return ticker

    except Exception as e:
        print(f"티커 조회 중 오류 발생: {stock_name}: {e}")

    # 종목이 존재하지 않으면 None 반환
    return None

In [3]:
# 드라이버 초기화 및 URL 열기
download_path = os.path.join(os.getcwd(), 'IR_pdf_raw')
os.makedirs(download_path, exist_ok=True)

# WebDriver 초기화
driver = init_driver(download_path)
wait = WebDriverWait(driver, 10)

try:
    driver.get('https://kind.krx.co.kr/corpgeneral/irschedule.do?method=searchIRScheduleMain&gubun=iRMaterials&marketType=2&kosdaqSegment=1')
    time.sleep(3)  # 페이지 로딩을 위한 대기
except Exception as e:
    print(f"페이지 로드 중 오류 발생: {e}")
    driver.quit()

# 페이지네이션 및 다운로드 루프
for i in range(1, 9):
    try:
        to_KOSPI = driver.find_element(By.ID, 'rWertpapier')
        to_KOSPI.click()

        set_timespan_all = driver.find_element(By.CLASS_NAME, 'ord-07')
        set_timespan_all.click()

        select_element = driver.find_element(By.TAG_NAME, "select")
        select = Select(select_element)
        select.select_by_value("100")  # 100개씩 보기

        pagination_wrap = driver.find_element(By.CLASS_NAME, 'paging')
        pagination_list = pagination_wrap.find_elements(By.TAG_NAME, 'a')
        pagination_list[i + 1].click()

        confirm = driver.find_element(By.CLASS_NAME, 'search-btn')
        confirm.click()

        time.sleep(3)  # 검색 결과 로딩 대기

        section = driver.find_element(By.CLASS_NAME, 'scrarea')
        rows = section.find_elements(By.TAG_NAME, 'tr')

        for j in range(1, len(rows)):
            try:
                row = rows[j]
                row_tds = row.find_elements(By.TAG_NAME, 'td')

                corp_name = row_tds[1].text.strip()
                date = row_tds[2].text.strip()

                btns = row_tds[-1].find_elements(By.TAG_NAME, 'a')
                btn = btns[-1]

                pdf_title = btn.get_attribute('onclick').split(',')[-3]
                if 'eng' in pdf_title.lower() or '영문' in pdf_title:
                    continue  # 영문 보고서는 건너뜁니다

                btn.click()
                time.sleep(5)  # 파일 다운로드 대기

                latest_file = get_latest_file(download_path)
                if latest_file:
                    stock_code = get_ticker_by_name(corp_name)
                    to_YYYY_MM = f'{date[:4]}.{date[5:7]}'
                    if stock_code:
                        new_dir_path = os.path.join(download_path, stock_code, to_YYYY_MM)
                        new_file_name = f"{stock_code}_{corp_name}_{date}_IR.pdf"
                    else:
                        new_dir_path = os.path.join(download_path, 'unknown', to_YYYY_MM)
                        new_file_name = f"unknown_{corp_name}_{date}_IR.pdf"

                    os.makedirs(new_dir_path, exist_ok=True)
                    new_file_path = os.path.join(new_dir_path, new_file_name)
                    os.rename(latest_file, new_file_path)
                    print(f"Renamed {os.path.basename(latest_file)} to {new_file_name}")
            except (NoSuchElementException, StaleElementReferenceException, TimeoutException) as e:
                print(f"{j}번째 row 처리 중 오류 발생: {e}")
                continue
            except Exception as e:
                print(f"파일 저장 중 오류 발생: {e}")
                continue

    except (NoSuchElementException, StaleElementReferenceException, TimeoutException) as e:
        print(f"{i}번째 페이지 처리 중 오류 발생: {e}")
        continue
    except Exception as e:
        print(f"페이지 처리 중 예상치 못한 오류 발생: {e}")
        continue

# 브라우저 종료
driver.quit()

종목명:  솔루엠
Renamed 솔루엠 24' IR Book.pdf to 248070_솔루엠_2024-08-19_IR.pdf
종목명:  크래프톤
Renamed KRAFTON-2Q24 Earnings Release_vF_KOR.pdf to 259960_크래프톤_2024-08-13_IR.pdf
종목명:  크래프톤
Renamed KRAFTON-2Q24 Earnings Release_vF_KOR.pdf to 259960_크래프톤_2024-08-12_IR.pdf
종목명:  케이카
Renamed K Car_IR PT_KR_20240802.pdf to 381970_케이카_2024-08-08_IR.pdf
종목명:  엘앤에프
Renamed L&F '24년 2분기 실적발표_(KIND,홈페이지게시).pdf to 066970_엘앤에프_2024-08-06_IR.pdf
종목명:  케이카
Renamed K Car_IR PT_KR_20240802.pdf to 381970_케이카_2024-08-05_IR.pdf
종목명:  케이카
Renamed 케이카 실적발표 PT_국문_Q2'24_F.pdf to 381970_케이카_2024-08-01_IR.pdf
종목명:  LIG넥스원
Renamed [LIG넥스원]24년 2분기 영업(잠정)실적(KOR).pdf to 079550_LIG넥스원_2024-07-26_IR.pdf
종목명:  포스코DX
Renamed 240725_포스코DX IR자료_2Q24년.pdf to 022100_포스코DX_2024-07-25_IR.pdf
종목명:  롯데케미칼
Renamed ★240704_CEO_Investor_Day_vff_released.pdf to 011170_롯데케미칼_2024-07-04_IR.pdf
종목명:  사조대림
Renamed 사조대림 IR자료_2024.06.27.pdf to 003960_사조대림_2024-06-27_IR.pdf
종목명:  케이카
Renamed K Car_IR PT_KR_20240520.pdf to 381970_케이카_2024-06-27_IR.pdf


MaxRetryError: HTTPConnectionPool(host='localhost', port=51176): Max retries exceeded with url: /session/b964e7ea2691abddb29e56605910e566/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11766f970>: Failed to establish a new connection: [Errno 61] Connection refused'))